## Generate BEV images

In [ ]:
"""
Python Point Cloud BEV Generation
Written by Petros626
Source by MacCallister Higgins (https://github.com/mjshiggins/ros-examples)

BEVDetNet paper parameters
cell (voxel) size: 0.1 m
image size: 512x256x3 (Valeo created model architecture)
channel R: height 
channel G: intensity
channel B: density (occupancy)
pixel values z-coords.: [-2.73, 1.27]
offset: [2.73]
point cloud range: [0, -30, -2.73, 60, 30, 1.27] (implemented with OpenPCDet)
"""

from numpy import float32, ndarray
from numpy.typing import NDArray
from pathlib import Path


box_colormap = [
    [255, 255, 255], # not assigned
    [0, 255, 0], # Car Green
    [255, 0, 255], # Pedestrian Violet
    [0, 255, 255], # Cyclist Yellow
]

class_names = { 
    1: 'Car', 
    2: 'Ped.',
    3: 'Cyc.'
}

# ==============================================================================
# SET PARAMETERS FOR BEV CONFIGURATION
# ==============================================================================
class Config:
    """Default configuration values for BEV generation."""
    IMAGE_WIDTH: int = 640 # YOLOv8 OBB 640x640 
    IMAGE_HEIGHT: int = 640 
    OFFSET_LIDAR: float = 2.73 # Offset from ground level, shifting the LiDAR height values [m]
    Z_MIN_HEIGHT: float = -2.73 # Minimum height for z-axis in the LiDAR point cloud [m]
    Z_MAX_HEIGHT: float = 1.27 # Maximum height for z-axis in the LiDAR point cloud [m]
    OUT_MIN: int = 0 # Minimum pixel value for h,i,d representation in the output BEV image
    OUT_MAX: int = 255 # Maximum pixel value for height representation in the output BEV image
    SAVE_IMG_TO_DISK: bool = True # Save image flag
    SAVE_LBL_TO_DISK: bool = True # Save label flag (only for mode train|val)
    NORM_BOX_COORDS: bool = True # Normalize Box corners between [0...1]
    CELL_SIZE: float = 0.1 # Cell resolution for the BEV grid [m], determines the granularity of the generated BEV image
    OUTPUT_PATH_DEF: str = 'bev_images' # Save path without categorization
    OUTPUT_PATH_TRAIN: str = 'bev_images/train/imgs'#'bev_images/train/imgs' # Save path for training data
    OUTPUT_PATH_TRAIN_LABELS: str = 'bev_images/train/labels'#'bev_images/train/labels' # Save path for training labels 
    OUTPUT_PATH_VAL: str = 'bev_images/val/imgs' # Save path for validation data
    OUTPUT_PATH_VAL_LABELS: str = 'bev_images/val/labels' # Save path for validation labels
    WIN_NAME_RGB: str = 'BEV Generator RGB' # Preview Window
    KITTI_BIN_PATH: str = '/home/rlab10/OpenPCDet/data/kitti/training/velodyne/000000.bin' # raw data single file
    KITTI_PATH: str = '/home/rlab10/OpenPCDet/data/kitti/data_test_pipeline/lidar_bev/test_several' # raw data several files
    PKL_TRAIN: str = '/home/rlab10/OpenPCDet/data/kitti/kitti_train_dataset.pkl' # train data augmented
    #PKL_TRAIN: str = '/home/rlab10/OpenPCDet/data/kitti/data_test_pipeline/visualizer/kitti_train_dataset.pkl' # test pipeline data
    PKL_VAL: str = '/home/rlab10/OpenPCDet/data/kitti/kitti_val_dataset.pkl' # pre-processed*
    PROGRESS_TRAIN_FILE: str = "/home/rlab10/OpenPCDet/lidar2bev/bev_train_progress.json" 
    PROGRESS_VAL_FILE: str = '/home/rlab10/OpenPCDet/lidar2bev/bev_val_progress.json'

    # *points/boxes in point cloud range, FOV points only, PFE (absolute_coordinates_encoding), DP (mask_points_and_boxes_outside_range, shuffe_points)

class BEVGenerator:
    def __init__(self, config: Config):
        from cv2 import IMWRITE_PNG_COMPRESSION
        from numpy import zeros, full, uint8

        self.config = config
        # `lowest` will store the lowest observed height value in the LiDAR data
        # `fnameCounter` will be used to generate unique filenames for saving output images
        # `base_filename_img` will be the name of the generated output images
        # `extension_img` will be the image format
        self.lowest, self.fnameCounter = 0.0, 0
        self.base_filename_img = f'bev_'
        self.extension_img = '.png'
        self.extension_lbl = '.txt'

        # 2D arrays for storing height, intensity, and density values at each pixel
        self.heightArray = full((self.config.IMAGE_HEIGHT, self.config.IMAGE_WIDTH), float('-inf'), dtype=float)
        self.intensityArray = zeros((self.config.IMAGE_HEIGHT, self.config.IMAGE_WIDTH), dtype=float)
        self.densityArray = zeros((self.config.IMAGE_HEIGHT, self.config.IMAGE_WIDTH), dtype=int)
        # Initialize the heightmap (a 3-channel BGR image) to store the visual representation of height data in color
        self.heightmap_bgr = zeros((config.IMAGE_HEIGHT, config.IMAGE_WIDTH, 3), dtype=uint8)

        # Ensure the output directory exists (create it if necessary)
        Path(config.OUTPUT_PATH_TRAIN).mkdir(parents=True, exist_ok=True)
        Path(config.OUTPUT_PATH_TRAIN_LABELS).mkdir(parents=True, exist_ok=True)
        Path(config.OUTPUT_PATH_VAL).mkdir(parents=True, exist_ok=True)
        Path(config.OUTPUT_PATH_VAL_LABELS).mkdir(parents=True, exist_ok=True)

        # Define compression parameters for saving PNG images
        # 0 means no compression (fastest but large file size), and 9 is maximum compression (slowest but smallest file size)
        self.compression_params = [IMWRITE_PNG_COMPRESSION, 0]

        # MAX_INTENSITY corresponds to the maximum intensity value for a LiDAR point. The KITTI dataset uses an intensity range [0...255], which is then mapped to [0...1] for normalization.
        self.MAX_INTENSITY = 1.0 
        # MAX_DENSITY sets the maximum expected density of LiDAR points per pixel in the BEV grid.
        # The value is capped at 12.0, meaning no pixel will have more than 10 LiDAR points by default.
        self.MAX_DENSITY = 12.0  # below in the code, there is a test code.

    # ==============================================================================
    # CONVERTS 3D POINTCLOUD TO BEV IMAGE (WITH) LABELS
    # ==============================================================================
    def lidar2bev(self, cloud: NDArray[float32], gt_boxes: ndarray, mode: str = None, frame_id: str = None, augment_idx: int = 0, val_info: dict = None, vis: bool = True) -> None:
        from copy import deepcopy
        from numpy import int32, array, savetxt
        from pcdet.utils.bev_utils import map_height2channel, map_intensity2channel, map_density2channel, map_pc2rc, map_rc2pc, normalize_coordinates, max_density_distribution
        from tools.visual_utils.bev_vis_utils import draw_bbox_arrow, draw_bbox_direction, draw_bbox_keypoint, get_rot_bevbox, draw_lbl_and_score, draw_lbl_nl_score
        from cv2 import namedWindow, startWindowThread, WINDOW_AUTOSIZE, cvtColor, imshow, imwrite, COLOR_BGR2RGB, waitKey, polylines

        # Reset arrays
        self.lowest, self.heightArray[:, :] = float('inf'), float('-inf')
        self.intensityArray[:, :], self.densityArray[:, :] = 0.0, 0.0
        #print('BEVGenerator: Point Cloud Received')

        """cloud: [0] = x, [1] = y,  [2] = z, [3] = intensity"""
        
        #print(f'BEVGenerator: Loaded point cloud with {len(cloud)} points')
    
        row, column = [0], [0] # Pass by reference simulation

        # Loop through points and populate the grid
        for j in range(len(cloud)):
            #  Map LiDAR coordinates to image grid and check bounds 
            if(map_pc2rc(cloud[j][0], cloud[j][1], row, column, self.config.IMAGE_HEIGHT, self.config.IMAGE_WIDTH, self.config.CELL_SIZE) == 1 and 
               row[0] >= 0 and row[0] < self.config.IMAGE_HEIGHT and column[0] >= 0 and 
               column[0] < self.config.IMAGE_WIDTH):
                
                # Update max height and density
                if(cloud[j][2] > self.heightArray[row[0]][column[0]]):
                    self.heightArray[row[0]][column[0]] = cloud[j][2]
                    # Increment the number of points in the cell
                    self.densityArray[row[0]][column[0]] += 1
                
                # get max intensity (tested, intensity too bright)
                # source: https://github.com/bostondiditeam/MV3D/blob/540a3f85ceec8aa6cc51613bc8961c980c13208e/utils/bag_to_kitti/lidar/src/lidar/src/lidar_node.cpp
                # this approach is different from BirdNet: a 3D Object Detection Framework from LiDAR information and BirdNet+: End-to-End 3D Object Detection in LiDAR Bird's Eye View
                #if(cloud[j][3] > self.intensityArray[row[0]][column[0]]):
                #    self.intensityArray[row[0]][column[0]] = cloud[j][3]

                # Track the lowest point for flood fill
                elif (cloud[j][2] < self.lowest):
                    self.lowest = cloud[j][2]

                # Accumulate intensity values to later compute the mean
                self.intensityArray[row[0]][column[0]] += cloud[j][3] # intensity value

        # Old code, slower, better to understand
        # for i in range(self.config.IMAGE_HEIGHT):
        #     for j in range(self.config.IMAGE_WIDTH):
        #         if self.densityArray[i][j] > 0:
        #             self.intensityArray[i][j] /= self.densityArray[i][j]
        # New code: Compute mean intensity
        valid_density_mask = self.densityArray > 0
        self.intensityArray[valid_density_mask] /= self.densityArray[valid_density_mask]

        #############################
        # Test code for max density #
        #############################
        #max_density_distribution(density_array=self.densityArray, image_height=self.config.IMAGE_HEIGHT, image_width=self.config.IMAGE_WIDTH, distribution_bins=10)

        # Create OpenCV image for visualization
        x, y = [0.0], [0.0] # pass by reference simulation
      
        # Loop through the image grid
        for i in range(self.config.IMAGE_HEIGHT):
            for j in range(self.config.IMAGE_WIDTH):
                # Convert image grid coordinates to LiDAR point cloud coordinates
                map_rc2pc(x, y, i, j, self.config.IMAGE_HEIGHT, self.config.IMAGE_WIDTH, self.config.CELL_SIZE)
                
                # If valid height data exists, update pixel values
                if self.heightArray[i][j] > float('-inf'):
                    # Set height (blue), intensity (green), and density (red) in BEV image
                    self.heightmap_bgr[i, j] = [
                        map_height2channel(self.heightArray[i][j], self.config.OFFSET_LIDAR, self.config.Z_MIN_HEIGHT, self.config.Z_MAX_HEIGHT, self.config.OUT_MIN, self.config.OUT_MAX), # B = height
                        map_intensity2channel(self.intensityArray[i][j], self.MAX_INTENSITY), # G = intensity
                        map_density2channel(self.densityArray[i][j], self.MAX_DENSITY) # R = density
                    ]
                else:
                    # Set black pixels for no data
                    self.heightmap_bgr[i, j] = [0, 0, 0]

        # Convert from BGR to RGB color format for visualization
        self.heightmap_rgb = cvtColor(self.heightmap_bgr, COLOR_BGR2RGB)
        heightmap_rgb_vis = deepcopy(self.heightmap_rgb)
   
        if gt_boxes is not None:
            label_data = []
            
            #for idx, box in gt_boxes:
            for idx, box in enumerate(gt_boxes):
                x, y, l, w, yaw, cls = [box[i] for i in [0, 1, 3, 4, 6, 7]]
                result = get_rot_bevbox(x, y, l, w, yaw, cls, heightmap_rgb_vis, bev_res=self.config.CELL_SIZE)
                
                # Skip invalid boxes
                #if isinstance(result, ndarray) and any(result == -1):
                if any(val == -1 for val in result[1:9]):
                    continue
                # Unpack the result (only if the box is valid)
                cls, x1, y1, x2, y2, x3, y3, x4, y4, bbox_color, centroid = result

                polygon = array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], dtype=int32).reshape((-1, 1, 2))
                # Draw rotated BEV box
                polylines(heightmap_rgb_vis, [polygon], isClosed=True, color=bbox_color, thickness=1)

                # Draw visualization elements...
                #draw_lbl_and_score(heightmap_rgb_vis, class_names.get(cls, 'Unknown'), conf_score=None, centroid=centroid, color=bbox_color)
                #draw_lbl_nl_score(heightmap_rgb_vis, class_names.get(cls, 'Unknown'), conf_score=None, centroid=centroid, yaw=yaw, color=bbox_color)
                #draw_bbox_direction(heightmap_rgb_vis, array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]]))
                #draw_bbox_arrow(heightmap_rgb_vis, centroid=centroid, yaw=yaw, cls=cls, bev_res=self.config.CELL_SIZE, color=bbox_color)
                #draw_bbox_keypoint(heightmap_rgb_vis, centroid=centroid, cls=cls, bev_res=self.config.CELL_SIZE)

                if self.config.NORM_BOX_COORDS: # Yolov8 OBB format: class_index x1 y1 x2 y2 x3 y3 x4 y4
                    #print('Normalizing BEV box coords to [0...1]')
                    x1, y1, x2, y2, x3, y3, x4, y4 = normalize_coordinates(x1, y1, x2, y2, x3, y3, x4, y4, heightmap_rgb_vis.shape[1], heightmap_rgb_vis.shape[0])
                    
                base_label = f'{int(cls)} {x1:.6f} {y1:.6f} {x2:.6f} {y2:.6f} {x3:.6f} {y3:.6f} {x4:.6f} {y4:.6f}'
                
                if mode == 'val' and val_info is not None:
                    label_line = (f'{base_label} '
                                f'{val_info["heights"][idx]:.2f} '
                                f'{val_info["truncated"][idx]:.2f} '
                                f'{int(val_info["occluded"][idx])}') 
                else:   
                    label_line = base_label
                label_data.append(label_line)
                    
        if vis:
            # Display the generated BEV image
            namedWindow(self.config.WIN_NAME_RGB, WINDOW_AUTOSIZE)
            #startWindowThread()
            waitKey(1) # BUGFIX: temporary
            imshow(self.config.WIN_NAME_RGB, heightmap_rgb_vis) # BUG: frame does not update in preview, only with diff. sample

        #print(f'BEVGenerator: BEV image {heightmap_rgb_vis.shape} is generated.')
        
        if mode == 'train':
            if self.config.SAVE_IMG_TO_DISK == True:
                #filename_img = Path(self.config.OUTPUT_PATH_TRAIN) / f'bev_{frame_id}_{augment_idx}{self.extension_img}'
                filename_img = Path(self.config.OUTPUT_PATH_TRAIN) / f'bev_{frame_id}{self.extension_img}'
                imwrite(filename_img, self.heightmap_rgb, self.compression_params)
                print(f'BEVGenerator: Saving image {self.heightmap_rgb.shape[:2]} to: {filename_img}')

            if self.config.SAVE_LBL_TO_DISK == True:
                #filename_lbl = Path(self.config.OUTPUT_PATH_TRAIN_LABELS) / f'bev_{frame_id}_{augment_idx}{self.extension_lbl}'
                filename_lbl = Path(self.config.OUTPUT_PATH_TRAIN_LABELS) / f'bev_{frame_id}{self.extension_lbl}'
                savetxt(filename_lbl, array(label_data), fmt='%s')
                print(f'BEVGenerator: Saving labels to: {filename_lbl}')

        elif mode == 'val':
            if self.config.SAVE_IMG_TO_DISK == True:
                filename_img = Path(self.config.OUTPUT_PATH_VAL) / f'bev_val_{frame_id}{self.extension_img}'
                imwrite(str(filename_img), self.heightmap_rgb, self.compression_params)
                print(f'BEVGenerator: Saving image {self.heightmap_rgb.shape[:2]} to: {filename_img}')

            if self.config.SAVE_LBL_TO_DISK == True:
                filename_lbl = Path(self.config.OUTPUT_PATH_VAL_LABELS) / f'bev_val_{frame_id}{self.extension_lbl}'
                #with open(filename_lbl, 'w')as f:
                #    f.write('\n'.join(label_data))
                savetxt(filename_lbl, array(label_data), fmt='%s')
                print(f'BEVGenerator: Saving labels to: {filename_lbl}')

        else:
            if self.config.SAVE_IMG_TO_DISK == True:
                # Find a unique filename
                while 1:
                    filename = Path(self.config.OUTPUT_PATH_DEF) / f'{self.base_filename_img}{self.fnameCounter:06d}{self.extension_img}'
                    if not filename.exists():
                        break
                    self.fnameCounter += 1
                
                # Save the image
                imwrite(str(filename), self.heightmap_rgb, self.compression_params)
                print(f'BEVGenerator: Saving image {self.heightmap_rgb.shape[:2]} to: {filename}')

# ==============================================================================
# MAIN FUNCTION
# ==============================================================================
def main(mode=str):
    from traceback import format_exc
    from pcdet.utils.bev_utils import load_kitti_pointcloud, save_progress, load_progress, get_unique_frame_id
    from cv2 import waitKey, destroyAllWindows
    from collections import defaultdict
    

    config = Config()
    bev_generator =  BEVGenerator(config)
   
    if mode == 'single':
        try:
            print('Starting LiDAR_BEV Script\n')
            print('Parameters for BEV generation:')
            print('image_height: ', config.IMAGE_HEIGHT)
            print('image_width: ', config.IMAGE_WIDTH)
            print('offset_lidar: ', config.OFFSET_LIDAR)
            print('z_min_height: ', config.Z_MIN_HEIGHT)
            print('z_max_height: ', config.Z_MAX_HEIGHT)
            print('out_min: ', config.OUT_MIN)
            print('out_max: ', config.OUT_MAX)
            print('save_img_to_disk: ', config.SAVE_IMG_TO_DISK)
            print('cell_size: ', config.CELL_SIZE)
            print('output_path: ', config.OUTPUT_PATH_DEF)
            print('kitti_bin_path:', config.KITTI_BIN_PATH)
            print('\n')

            #@jit(target_backend='cuda', forceobj=True) # GPU
            #def process_bev(): 
            # Load and pre-process single point cloud
            point_cloud = load_kitti_pointcloud(config.KITTI_BIN_PATH, None, 0, 0)

            # Generate BEV image
            bev_generator.lidar2bev(cloud=point_cloud, gt_boxes=None, mode=None, frame_id=0, augment_idx=None, vis=True)

            #exec_time = timeit(process_bev, number=1)
            #print(f'Execution time for loading point cloud and generating BEV: {exec_time:.4f} seconds') 

            print('Press q to quit.')

            while 1:
                if waitKey(1) & 0xFF == ord('q'):
                    break
            
        except FileNotFoundError:
            print(f'Error: Could not find point cloud file at {config.KITTI_BIN_PATH}')
        
        except Exception as e:
            error = format_exc()
            print(f'Error occured: {error}')
        finally:
            destroyAllWindows()

    elif mode == 'several':
        try:
            print('Starting LiDAR_BEV Script\n')
            print('Parameters for BEV generation:')
            print('image_height: ', config.IMAGE_HEIGHT)
            print('image_width: ', config.IMAGE_WIDTH)
            print('offset_lidar: ', config.OFFSET_LIDAR)
            print('z_min_height: ', config.Z_MIN_HEIGHT)
            print('z_max_height: ', config.Z_MAX_HEIGHT)
            print('out_min: ', config.OUT_MIN)
            print('out_max: ', config.OUT_MAX)
            print('save_img_to_disk: ', config.SAVE_IMG_TO_DISK)
            print('cell_size: ', config.CELL_SIZE)
            print('output_path: ', config.OUTPUT_PATH_DEF)
            print('kitti_path:', config.KITTI_PATH)
            print('\n')
            
            # Load and pre-process several point clouds
            point_clouds = load_kitti_pointcloud(config.KITTI_PATH, None, 0, 0) 
            
            #@jit(target_backend='cuda', forceobj=True) # GPU
            #def process_bev_multiple():
            for point_cloud in point_clouds:
                
                # Generate BEV images
                bev_generator.lidar2bev(cloud=point_cloud, gt_boxes=None, frame_id=None, augment_idx=0, val_info=None, vis=True)

                print('Press q to quit diashow.\n')

                if waitKey(1) & 0xFF == ord('q'):
                    print('Interrupt')
                    break

            #execution_time = timeit(process_bev_multiple, number=1)        
            #print(f'Execution time for loading multiple point clouds and generating BEVs: {execution_time:.4f} seconds')
              
        except FileNotFoundError:
            print(f'Error: Could not find point cloud files at {config.KITTI_PATH}')
            #print(f'Error: Could not find point cloud file at {config.PKL_DATA_PATH}')
        except Exception as e:
            error = format_exc()
            print(f'Error occured: {error}')
        finally:
            destroyAllWindows()
        
    elif mode == 'train':
        try:
            print('Starting LiDAR_BEV Script\n')
            print('Parameters for BEV generation:')
            print('image_height: ', config.IMAGE_HEIGHT)
            print('image_width: ', config.IMAGE_WIDTH)
            print('offset_lidar: ', config.OFFSET_LIDAR)
            print('z_min_height: ', config.Z_MIN_HEIGHT)
            print('z_max_height: ', config.Z_MAX_HEIGHT)
            print('out_min: ', config.OUT_MIN)
            print('out_max: ', config.OUT_MAX)
            print('save_img_to_disk: ', config.SAVE_IMG_TO_DISK)
            print('cell_size: ', config.CELL_SIZE)
            print('output_path imgs: ', config.OUTPUT_PATH_TRAIN)
            print('output_path lbls: ', config.OUTPUT_PATH_TRAIN_LABELS)
            print('pkl_train:', config.PKL_TRAIN)
            print('\n')

            last_sample_idx, last_augment_idx, frame_id_dict = load_progress(progress_file=config.PROGRESS_TRAIN_FILE)
            print(f"Resuming from Sample {last_sample_idx}, Augmentation {last_augment_idx}")

            # Load and pre-process several point clouds
            _, _, num_samples, _ = load_kitti_pointcloud(None, config.PKL_TRAIN, 0, 0)
            num_augmentations = 5 # original, gt_sampling, world flip, world rotation, local rotation

            for sample_idx in range(last_sample_idx, num_samples):
                for augment_idx in range(last_augment_idx, num_augmentations):
    
                    point_cloud, gt_boxes,  _, curr_frame_id = load_kitti_pointcloud(None, config.PKL_TRAIN, sample_idx, augment_idx)
                    base_frame_id, frame_id_dict = get_unique_frame_id(curr_frame_id, augment_idx, frame_id_dict)

                    print(f'Processing Frame_ID:{curr_frame_id}, Sample:{sample_idx}, Augmentation:{augment_idx}')
                    bev_generator.lidar2bev(cloud=point_cloud, gt_boxes=gt_boxes, mode='train', frame_id=f"{base_frame_id}_{augment_idx}", augment_idx=augment_idx, vis=False)

                    save_progress(sample_idx=sample_idx, augment_idx=augment_idx, frame_id_dict=frame_id_dict, progress_file=config.PROGRESS_TRAIN_FILE)

                    print('Press q to quit diashow.\n')
                   
                    #if waitKey(1) & 0xFF == ord('q'):
                    #    print('Interrupt')
                    #    break

                last_augment_idx = 0

            if sample_idx == num_samples-1 and augment_idx == num_augmentations - 1:
                save_progress(sample_idx=0, augment_idx=0, frame_id_dict=defaultdict(int), progress_file=config.PROGRESS_TRAIN_FILE)
                print("All samples processed. Progress reset to 0.")

            # Code to generate the replaced only
            # frame_to_sample_map = {'003381': 8,    '004449': 13,   '000129': 44,   '004272': 131, '000017': 161, '002480': 164,  '007443': 204,  '003261': 265,  '003842': 269,  
            #                        '003714': 271,  '003586': 277,  '004464': 279,  '002872': 363,  '002226': 390,  '005543': 466, '001209': 478,  '002072': 558,  '002658': 596,  '005253': 639,  
            #                        '002110': 694, '003089': 978,  '000872': 995,  '004857': 1019, '000070': 1034, '001990': 1050, '007106': 1085, '003732': 1103,'003415': 1113, '005608': 1122,
            #                        '001032': 1145, '005187': 1211, '002912': 1238, '002829': 1254, '007101': 1333, '002396': 1370, '002358': 1382, '001830': 1425, '002549': 1454, '002444': 1471, '007021': 1526, 
            #                        '003270': 1552,'003995': 1591, '006055': 1596, '004088': 1619, '005815': 1693, '002776': 1703, '005025': 1723, '001033': 1766, '007444': 1812,'004525': 1845, '006621': 1881, 
            #                        '007237': 1954, '006940': 2001, '005177': 2187, '005173': 2219, '006648': 2229, '000815': 2298, '004355': 2325, '002475': 2346, '003760': 2349, '001529': 2471, '005943': 2525, 
            #                        '002146': 2602, '001759': 2606, '003244': 2639, '004159': 2640, '007399': 2714,'001730': 2728,'001493': 2731,'002452': 2818, '006654': 2855, 
            #                        '002221': 2900, '004572': 2959, '001482': 3055, '005561': 3059, '003565': 3085, '004855': 3090,'006064': 3111, '002671': 3114, '002410': 3282, '002852': 3311, '007379': 3360, 
            #                        '000846': 3424, '000672': 3437, '000763': 3453, '006193': 3457, '007285': 3509, '003363': 3527, '002954': 3557, '000640': 3588, '001971': 3609, '003989': 3649,'000853': 3708,}

            # _, _, num_samples, _ = load_kitti_pointcloud(None, config.PKL_TRAIN, 0, 0)
            # num_augmentations = 5 

            # for frame_id, sample_idx in frame_to_sample_map.items():
            #     for augment_idx in range(num_augmentations):

            #         point_cloud, gt_boxes, _, curr_frame_id = load_kitti_pointcloud(None, config.PKL_TRAIN, sample_idx, augment_idx)

            #         print(f'Processing Frame_ID: {curr_frame_id}, Sample: {sample_idx}, Augmentation: {augment_idx}')

            #         bev_generator.lidar2bev(
            #             cloud=point_cloud,
            #             gt_boxes=gt_boxes,
            #             mode='train',
            #             frame_id=f"{curr_frame_id}_{augment_idx}",
            #             augment_idx=augment_idx,
            #             vis=False
            #         )

            #         print('\n')

        except FileNotFoundError:
            print(f'Error: Could not find .pkl file at {config.PKL_TRAIN}')
        except Exception as e:
            error = format_exc()
            print(f'Error occured: {error}')
        finally:
            destroyAllWindows()

    elif mode == 'val':
        try:
            print('Starting LiDAR_BEV Script\n')
            print('Parameters for BEV generation:')
            print('image_height: ', config.IMAGE_HEIGHT)
            print('image_width: ', config.IMAGE_WIDTH)
            print('offset_lidar: ', config.OFFSET_LIDAR)
            print('z_min_height: ', config.Z_MIN_HEIGHT)
            print('z_max_height: ', config.Z_MAX_HEIGHT)
            print('out_min: ', config.OUT_MIN)
            print('out_max: ', config.OUT_MAX)
            print('save_img_to_disk: ', config.SAVE_IMG_TO_DISK)
            print('cell_size: ', config.CELL_SIZE)
            print('output_path imgs: ', config.OUTPUT_PATH_VAL)
            print('output_path lbls: ', config.OUTPUT_PATH_VAL_LABELS)
            print('pkl_val:', config.PKL_VAL)
            print('\n')

            last_sample_idx, _, lidar_idx_dict = load_progress(progress_file=config.PROGRESS_TRAIN_FILE)
            print(f"Resuming from Sample {last_sample_idx}")

            _, _, _, num_samples, _ = load_kitti_pointcloud(None, config.PKL_VAL, 0, 0)
       
            for sample_idx in range(last_sample_idx, num_samples):
                
                point_cloud, gt_boxes, val_info, _, lidar_idx = load_kitti_pointcloud(None, config.PKL_VAL, sample_idx, 0)
                base_lidar_idx, lidar_idx_dict = get_unique_frame_id(lidar_idx, 4, lidar_idx_dict)

                print(f'Processing LiDAR_IDX:{lidar_idx}, Sample:{sample_idx}')
                
                bev_generator.lidar2bev(cloud=point_cloud, gt_boxes=gt_boxes, mode='val', frame_id=f"{base_lidar_idx}", augment_idx=0, val_info=val_info, vis=False)

                print('Press q to quit diashow.\n')

                save_progress(sample_idx=sample_idx, augment_idx=0, frame_id_dict=lidar_idx_dict, progress_file=config.PROGRESS_VAL_FILE)

                if waitKey(1) & 0xFF == ord('q'):
                        print('Interrupt')
                        break
                 
            if sample_idx == num_samples-1:
                save_progress(sample_idx=0, augment_idx=0, frame_id_dict=defaultdict(int), progress_file=config.PROGRESS_TRAIN_FILE)
                print("All samples processed. Progress reset to 0.")   

        except FileNotFoundError:
            print(f'Error: Could not find .pkl file at {config.PKL_VAL}')
        except Exception as e:
            error = format_exc()
            print(f'Error occured: {error}')
        finally:
            destroyAllWindows()

if __name__ == '__main__':
    #main('single')
    #main('several')
    #main('train') # TODO: solve bug with actualization of same frame_id -> different augmentations not appearing, implement saving the label in yolo format
    main('val')

In [ ]:
import pickle

path = '/home/rlab10/OpenPCDet/data/kitti/kitti_train_dataset.pkl'

replaced_frame_ids = [
    '003381', '004449', '000129', '004272', '000017', '002480', '007443', '003261', 
    '003842', '003714', '003586', '004464', '002872', '002226', '005543', '001209', 
    '002072', '002658', '005253', '002110', '003089', '000872', '004857', '000070', 
    '001990', '007106', '003732', '003415', '005608', '001032', '005187', '002912', 
    '002829', '007101', '002396', '002358', '001830', '002549', '002444', '007021', 
    '003270', '003995', '006055', '004088', '005815', '002776', '005025', '001033', 
    '007444', '004525', '006621', '007237', '006940', '000872', '002829', '005177', 
    '005173', '006648', '000815', '004355', '002475', '003760', '001529', '005943', 
    '002146', '001759', '003244', '004159', '007399', '001730', '001493', '002452', 
    '006654', '002221', '004572', '001482', '005561', '003565', '004855', '006064', 
    '002671', '002410', '002852', '007379', '000846', '000672', '000763', '006193', 
    '007285', '003363', '002954', '000640', '001971', '003989', '000853'
]

with open(path, 'rb') as f:
    data = pickle.load(f)

#print('type is: ', type(data))
replaced_indices = []

for idx, sample in enumerate(data):
    frame_id = sample[0]['frame_id']
    #print(f"{frame_id}")
    if frame_id in replaced_frame_ids:
        replaced_indices.append(idx)

print(f"{len(replaced_frame_ids)}")

#for idx in replaced_indices:
#    print(idx)